<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# LangGraph Agents: Orchestrator–Worker Pattern

In this tutorial, we’ll build a multi-agent system using LangGraph's **Orchestrator–Worker pattern**, ideal for dynamically decomposing a task into subtasks, assigning them to specialized LLM agents, and synthesizing their responses.

This pattern is particularly well-suited when the structure of subtasks is unknown ahead of time—such as when writing modular code, creating multi-section reports, or conducting research. The **orchestrator** plans and delegates, while the **workers** each complete their assigned section.

We’ll also use **Phoenix** to trace and debug the orchestration process. With Phoenix, you can visually inspect which tasks the orchestrator generated, how each worker handled its section, and how the final output was assembled.

By the end of this notebook, you’ll learn how to:
- Use structured outputs to plan subtasks dynamically.
- Assign subtasks to LLM workers via LangGraph's `Send` API.
- Collect and synthesize multi-step LLM outputs.
- Trace and visualize orchestration using Phoenix.


In [ ]:
%pip install  "arize-phoenix==9.0.1" "arize-phoenix-otel>=0.8.0" llama-index-llms-openai openai gcsfs nest_asyncio langchain langchain-openai openinference-instrumentation-langchain

In [ ]:
!pip install langchain_openai langgraph langchain_community duckduckgo-search

In [ ]:
import getpass
import os

from langgraph.graph import END, START, StateGraph

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Configure Phoenix Tracing

Make sure you go to https://app.phoenix.arize.com/ and generate an API key. This will allow you to trace your Langgraph application with Phoenix.

In [ ]:
PHOENIX_API_KEY = getpass.getpass("Phoenix API Key:")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="Orchestrator", auto_instrument=True)

Orchestrator‑Workers • Research‑Paper Generator
----------------------------------------------
The orchestrator plans research‑paper *subsections* (abstract, background …),
spawns one worker per subsection, then stitches everything into a full draft.

In [ ]:
import operator
from typing import Annotated, List, TypedDict

from IPython.display import Markdown
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.constants import Send

# Step 1: Defining the Planning Schema
To begin, we define a structured output schema using Pydantic. This schema ensures that the LLM returns well-formatted, predictable output when tasked with planning the structure of a research paper.

We create two models:

Subsection: Represents a single unit of the paper, including its name and a brief description of what it should cover.

Subsections: A wrapper that holds a list of these units.

By using these models with LangGraph’s with_structured_output feature, we enforce that the orchestrator LLM returns an organized plan — rather than freeform text — that downstream nodes (worker LLMs) can reliably use.

This schema acts as the blueprint for the rest of the workflow.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Subsection(BaseModel):
    name: str = Field(description="Name for this subsection of the research paper.")
    description: str = Field(
        description="Concise description of the general subjects to be covered in this subsection."
    )


class Subsections(BaseModel):
    Subsections: List[Subsection] = Field(description="All subsections of the research paper.")

# Step 2: Set Up LLM and Tools
We initialize gpt-3.5-turbo as our base LLM and bind it to the Subsections schema to create the orchestrator. We also load a DuckDuckGo search tool to allow worker agents to enrich sections with live web data.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
orchestrator_llm = llm.with_structured_output(Subsections)
ddg_results_tool = DuckDuckGoSearchResults(output_format="dict", max_results=5)

# Step 3: Define Graph State
We define two state schemas:

State holds the overall research paper workflow, including the topic, planned subsections, completed text, and final output.

WorkerState captures the task assigned to each worker — a single subsection — and where their contributions are accumulated.

This shared state structure lets LangGraph coordinate work between the orchestrator and its worker agents.

In [ ]:
class State(TypedDict):
    topic: str  # Research‑paper topic
    subsections: List[Subsection]  # Planned subsections
    completed_subsections: Annotated[List[str], operator.add]
    final_paper: str  # Synthesised draft


class WorkerState(TypedDict):
    subsection: Subsection
    completed_subsections: Annotated[List[str], operator.add]

# Step 4: Define Nodes
We define three core nodes in the graph:

orchestrator: Dynamically plans the structure of the paper by generating a list of subsections using structured output.

subsection_writer: Acts as a worker that writes one full subsection in academic Markdown, using the provided description and scope.

synthesiser: Merges all completed subsections into a single cohesive draft, separating sections with visual dividers.

Each node contributes to a modular, scalable paper-writing pipeline — and with Phoenix tracing, you can inspect every generation step in detail.

In [ ]:
def orchestrator(state: State):
    """Plan the research‑paper subsections dynamically."""
    plan = orchestrator_llm.invoke(
        [
            SystemMessage(content="Generate a detailed subsection plan for a research paper."),
            HumanMessage(content=f"Paper topic: {state['topic']}"),
        ]
    )
    return {"subsections": plan.Subsections}


def subsection_writer(state: WorkerState):
    """Write a single subsection (markdown)."""
    sub = state["subsection"]
    response = llm.invoke(
        [
            SystemMessage(
                content=(
                    "You are drafting a research‑paper subsection. "
                    "Follow academic tone, include citations placeholders (e.g. [1]), "
                    "and use Markdown headings where appropriate."
                )
            ),
            HumanMessage(
                content=(
                    f"Subsection name: {sub.name}\n"
                    f"Description / scope: {sub.description}\n\n"
                    "Write the full subsection now."
                )
            ),
        ]
    )
    return {"completed_subsections": [response.content]}


def synthesiser(state: State):
    """Concatenate all finished subsections into the final paper draft."""
    full_paper = "\n\n---\n\n".join(state["completed_subsections"])
    return {"final_paper": full_paper}

# Step 5: Assign Workers Dynamically
This function uses LangGraph’s Send API to launch a separate subsection_writer worker for each planned subsection. By dynamically spawning one worker per section, the system scales flexibly based on the topic’s complexity.

This approach is ideal for research paper generation, where the number of sections is not known ahead of time — and Phoenix helps trace the output from each worker node independently.

In [ ]:
def assign_workers(state: State):
    """Launch one worker per planned subsection."""
    return [Send("subsection_writer", {"subsection": s}) for s in state["subsections"]]

# Step 6: Construct the LangGraph Workflow
Here, we build the full LangGraph pipeline using a StateGraph. The workflow begins with the orchestrator node (to plan subsections), dynamically routes work to subsection_writer nodes (via assign_workers), and then aggregates all outputs in the synthesiser node.

LangGraph’s conditional edges and Send API enable scalable parallelism — and with Phoenix tracing enabled, you can view how each section is created, tracked, and stitched together.

In [ ]:
builder = StateGraph(State)

builder.add_node("orchestrator", orchestrator)
builder.add_node("subsection_writer", subsection_writer)
builder.add_node("synthesiser", synthesiser)

builder.add_edge(START, "orchestrator")
builder.add_conditional_edges("orchestrator", assign_workers, ["subsection_writer"])
builder.add_edge("subsection_writer", "synthesiser")
builder.add_edge("synthesiser", END)

research_paper_workflow = builder.compile()

# Step 7: Run the Research Paper Generator
We now invoke the compiled LangGraph with a sample topic: “Scaling Laws for Large Language Models.” The orchestrator plans the outline, each worker drafts a subsection in parallel, and the synthesizer assembles the full paper.

With Phoenix integrated, every step is traced — from section planning to writing and synthesis — giving you full visibility into the execution flow and helping debug or refine outputs.

In [ ]:
state = research_paper_workflow.invoke({"topic": "Scaling Laws for Large Language Models"})

print("===== RESEARCH PAPER DRAFT =====\n")
Markdown(state["final_paper"])

# Step 8: Check out your traces in Phoenix!